# Thomeer-Used-to-Model-High-Pressure-Mercury-Injection-Data
In this repository we provide the python code used to Closure correct and model High Pressure Mercury Injection (HPMI) data using the Thomeer hyperbola.

## Introduction:
This GitHub repository uses python code to import High Pressure Mercury Injection (HPMI) Core data fron Excel and then use a Thomeer hyperbola to model the Thomeer Capillary Pressure parameters as shown below. Ed Clerke used a similar method in Excel with Solver to estimate his Thomeer parameters for each HPMI sample that went into the Rosetta Stone Arab D Carbonate Thomeer database. His Excel spreadsheet is readily available, and we will included as copy of this spreadsheet in this repository. We also used fminsearch in Matlab too. We have used these types of software to establish our own Reservoir Characterization Reservoir-Specific core calibration databases in the past for a reservoir-specific studies.

![HPMI_Image](HPMI.png)

## How it Works:
The following animated image illustrates how this software operates. We start with the original HPMI data. The first step is to locate the point on the HPMI curve that represents the point where real data begins and not the HPMI data representing surface conformance around the plug sample. We also select the Initial Displacement Pressure (Pd1) for the sample for the first pore system too. We find that most carbonate rocks have bi-modal pore throat distributions representing two pore systems as shown in the example below. The selection of this point is performed with just a click on the Graphical User Interface (GUI) where this occurs. 

For the second step we pick the point for the Bulk Volume porosity of the first pore system (BV1) as well as the Initial Displacement Pressure for the second pore system (Pd2).

The third step is to select the Total porosity for the HPMI data called BVtotal where the porosity of the second pore system is called BV2:

              BV2 = BVtotal - BV1

![HPMI_Image](Thomeer_Parameter_fitting.gif)

This program uses Scipy Optimize Curve_fit to estimate the appropriate Thomeer parameters necessary to model the HPMI data. The points selected from the GUIs are used to estimate boundary conditions for these estimations, and the estimations for this example are shown below:

            Thomeer Parameters Estimated from Imported HPMI Data:
            Pd1 = 8.67  ,  G1 = 0.54 , BV1 = 10.13
            Pd2 = 389.1 ,  G2 = 0.24 , BV2 = 4.8


## Reservoir Characterization Workflow:
We are using the HPMI data from just one sample for this example. Our objective will be to employ this program in Geolog as a python loglan. In Geolog we will read the Pc data in from the SCAL data stored in a well and write the results for each sample back to Geolog to build our sample-by-sample core calibration database. We would then use the carbonate characterization workflow as employed in our following GitHub repository but alter the workflow to employ our own new reservoir-specific calibration data for our reservoir characterization.

https://github.com/Philliec459/Geolog-Used-to-Automate-the-Characterization-Workflow-using-Clerkes-Rosetta-Stone-calibration-data

In the above workflows we have used hundreds of HPMI samples as calibration. In the image below we are showing the Porosity vs. Permeability cross plot in the upper left for all the calibration samples used for the Arab D reservoir; all colored by Petrophysical Rock Type. We select a small group of poro-perm samples, and the Pc curves from this small group of selected samples is then shown in the lower left. The black Pc curve is the upscaled Pc curve from the selected samples, and the black bars in the histograms are the median value for the selected samples. We would be using upscaled Pc curves that varies level by level in the well to model saturations because of the changing reservoir quality along the wellbore.

![Thomeer_Pc_and_Thomeer_Parameters.gif](Thomeer_Pc_and_Thomeer_Parameters2.gif)

## Modeling of Saturations Using Thomeer Capillary Pressure Parameters: 
Also, the following image shows one example from our modeling of saturations from Capillary Pressure vs. log analysis. The match is very good. I personally have performed this type of characterization on at least 30 huge carbonate oil fields in Saudi, and the results shown below are very typical.

![HPMI_Image](logsats.gif)

1 Clerke, E. A., Mueller III, H. W., Phillips, E. C., Eyvazzadeh, R. Y., Jones, D. H., Ramamoorthy, R., Srivastava, A., (2008) “Application of Thomeer Hyperbolas to decode the pore systems, facies and reservoir properties of the Upper Jurassic Arab D Limestone, Ghawar field, Saudi Arabia: A Rosetta Stone approach”, GeoArabia, Vol. 13, No. 4, p. 113-160, October 2008.

# Requirements:

In [1]:
#!/usr/bin/env python3
# python loglan
# When referencing Geolog variables in a Python script, names must always be lower case.

# Imports
#############import geolog
import numpy as np
import pandas as pd
from numpy import diff

#%matplotlib inline
#%matplotlib notebook
%matplotlib qt


import matplotlib.pyplot as plt
import openpyxl
import scipy as sp
import scipy.interpolate



## Read in Pc data from xlsx file of Pc vs. BVocc:

In [2]:
# =============================================================================
# # ===========================================================================
# # #-------------------------------------------------------------------------- 
# # #                Read Pc Data Spreadsheet 
# # #  
# # #             
# # #--------------------------------------------------------------------------
# # ===========================================================================
# =============================================================================
#read the file

'''
Uncomment out the File Names for the sample that you want to use:
'''
# Dual Porosity Sample:
file = r'./data/Pc_data_dual_porosity.xlsx'
no_pore_sys= 2

# Dual Porosity Sample hard to fit:
#file = r'./data/Pc_data_dual_porosity_badfit.xlsx'
#no_pore_sys= 2

# Single Pore System example:
#file = r'./data/Pc_data_single_porosity.xlsx'
#no_pore_sys= 1

Thomeer_Pc_data = pd.read_excel(file,index_col=False)



# We want to use a Sample Number on our final plot
sample_no = "AA"



Pc_import = Thomeer_Pc_data['Pc']
BVocc_import = Thomeer_Pc_data['BVocc']

'''
  Cubic Spline Fit of data
'''
for i in range( 0,  len(BVocc_import),1):
    if BVocc_import[i]<=0:
        BVocc_import[i]=0.01
    else:
        BVocc_import[i]=BVocc_import[i]

x = np.log10(Pc_import)
y = np.log10(BVocc_import)

spline_number = 400

# Interpolate the data using a cubic spline to "new_length" samples
new_length = spline_number
new_x = np.linspace(x.min(), x.max(), new_length)
new_y = sp.interpolate.interp1d(x, y, kind='cubic')(new_x)

BVocc_r = 10**new_y
Pc_r = 10**new_x


x_Pc=np.array(BVocc_r)
y_Pc=np.array(Pc_r)
# =============================================================================
# # ===========================================================================
# # #------------------------------------------------------------ 
# # #               
# # #     End of reading in Pc data if required or Geolog below
# # #------------------------------------------------------------
# # ===========================================================================
# =============================================================================



# GUI Input or Pc data for Closure Correction and Thomeer Estimates of BV1, Pd1, Pd2 and BVtotal:
## Closure Correct the BVocc array from our Pc data:
Most HPMI data has issues in the low pressure regime with the mercruy conforming around the plug sample. We need to remove this early data because this is not matrix. This is called a Closure Correction. 

We find this is best done with a combination of BVocc vs. Pc Crossplots in Log-Log and Linear-Log as show below:

![HPMI_Image](PickClosure.png)

However, for this example we are using Log-Log.

In [3]:
# =============================================================================
# # ===========================================================================
# # #--------------------------------------------------------------------------
# ##
# ##            Graphical Input of Closure Correction and Pd1 
# ##
# # #--------------------------------------------------------------------------
# =============================================================================
# =============================================================================
# =============================================================================
#             Graphical Input for Closure Correction and Pd1 
# =============================================================================
def tellme(s):
    print(s)
    plt.title(s, fontsize=16, fontweight="bold", color = 'green')
    plt.draw()


plt.clf()   #clear plot of other things

plt.figure(num=2, figsize=(10, 10))
#plt.ion()
plt.loglog(x_Pc, y_Pc  , 'g-*', linewidth=1, label='HPMI Data' )
#plt.semilogy(Closure_PTD, 2  , 'r-o', linewidth=3, label='Closure Correction from PTD' )
plt.xlabel('BVocc', color = 'green', fontsize=12, fontweight="bold")
plt.ylabel('Pc',    color = 'green', fontsize=12, fontweight="bold")
plt.xlim(30.0, 0.1)
plt.ylim(1, 100000)
plt.grid(True, which="both",ls="-")
plt.legend()

#Use pts array to store selected points
pts = []

while len(pts) < 1:
    tellme('Select Closure Correction and Pd1')
    pts = np.asarray(plt.ginput(1, timeout=3))

Closure = pts.item(0)
Pd1_pick = pts.item(1)

print()
print('Closure Correction =', round(Closure,2), ' and Pd1 =', round(Pd1_pick,1))
print()

plt.close('all')
# =============================================================================
#             End of Graphical Input for Closure Correction and Pd1 
# =============================================================================

Select Closure Correction and Pd1

Closure Correction = 0.42  and Pd1 = 9.0



### Apply Closure Corrections to BVocc:

In [4]:
# =============================================================================
#           Closure Correction of Pc data
# =============================================================================
x_Pc_nocc = np.array(BVocc_r)

BV1=np.max(x_Pc_nocc)

for i in range(0,spline_number,1):
    
    BVocc_r[i]=BVocc_r[i]-Closure
    
    if BVocc_r[i] < 0:
        BVocc_r[i] = 0.001
    else:
        BVocc_r[i] = BVocc_r[i]

x_Pc = np.array(BVocc_r)

# =============================================================================
#          End of Closure Correction for Pc data
# =============================================================================

print(len(x_Pc))

400


## Select a single point for BV1 & Pd2 Thomeer parameter estimates directly from the GUI:

In [5]:
# =============================================================================
#             Graphical Input for Pd2 and BV1
# =============================================================================

def tellme(s):
    print(s)
    plt.title(s, fontsize=16, fontweight="bold", color = 'blue')
    plt.draw()

plt.clf()   #clear plot of other things

plt.figure(num=2, figsize=(10, 10))

plt.loglog(x_Pc, y_Pc  , 'b-*', linewidth=1, label='HPMI Data' )
#plt.loglog(Closure, Pd1  , 'k-o', linewidth=1, label='Closure Correction and Pd1 Estimate' )
plt.xlabel('BVocc',color = 'blue', fontsize=12, fontweight="bold")
plt.ylabel('Pc',color = 'blue', fontsize=12, fontweight="bold")

plt.xlim(50.0,0.01)
plt.ylim(1, 100000)
plt.grid(True, which="both",ls="-")
plt.legend()

pts = []


while len(pts) < 1:
    tellme('Select BV1 and Pd2')
    pts = np.asarray(plt.ginput(1, timeout=3))

BV1 = pts.item(0)
Pd2 = pts.item(1)
G1  = 0.4

print()
print('BV1 =', round(BV1,2), ' and Pd2 =', round(Pd2,1))
print()

plt.close('all')  
# =============================================================================
#             End of Graphical Input for Pd2 and BV1
# =============================================================================
#BV1=7
#Pd2=300




Select BV1 and Pd2

BV1 = 7.39  and Pd2 = 426.5



## Select the single point for BVtotal from GUI if you have over two pore systems:
This is the asymtote of the Pc curve in the last pore system. 

In [6]:
# =============================================================================
#             Graphical Input for BVtotal
# =============================================================================
if no_pore_sys > 1: 
    def tellme(s):
        print(s)
        plt.title(s, fontsize=16, fontweight="bold", color = 'red')
        plt.draw()

    plt.clf()   #clear plot of other things

    plt.figure(num=2, figsize=( 10, 10))

    plt.loglog(x_Pc, y_Pc  , 'r-*', linewidth=1, label='HPMI Data' )
    #plt.loglog(Closure, Pd1  , 'k-o', linewidth=1, label='Closure Correction and Pd1 Estimate' )
    plt.loglog(BV1, Pd2  , 'k-o', linewidth=1, label='BV1 and Pd2 Estimate' )
    plt.xlabel('BVocc',color = 'red', fontsize=12, fontweight="bold")
    plt.ylabel('Pc',color = 'red', fontsize=12, fontweight="bold")

    plt.xlim(50.0,0.01)
    plt.ylim(1, 100000)
    plt.grid(True, which="both",ls="-")
    plt.legend()

    #Use pts array to store selected points
    pts = []


    while len(pts) < 1:
        tellme('Select BVtotal')
        pts = np.asarray(plt.ginput(1, timeout=3))

    BVtotal = pts.item(0)
    junk = pts.item(1)

    BV2 = BVtotal - BV1
    print('BV2 = ',BV2)

    G2=0.2

    print()
    print('BVtotal =', round(BVtotal,2))
    print()


    plt.close('all')  
    # =============================================================================
    #             End of Graphical Input of BVtotal
    # =============================================================================
    #BVtotal=10



Select BVtotal
BV2 =  3.4654015402054217

BVtotal = 10.85



## We use Scipy Curve_Fit to Estimate the Thomeer Capillary Pressure Parameters:

In [7]:
# =============================================================================
#             Scipy for Curve_fit of Pd1, G1 and BV1
# =============================================================================
from scipy.optimize import curve_fit
quality1  =  ""
quality2  =  ""
# =============================================================================
#             Scipy for Curve_fit for just Pd1, G1 and BV1
# =============================================================================

if no_pore_sys == 1: 

    # Find Pd1
    Pd1_min_element = min(min(np.where(BVocc_r>0.1)))
    Pd1=Pc_r[Pd1_min_element]
     
    
    bvarray_pore1 = []; #make list of 0 length
    pcarray_pore1 = []

    for i in range(0, spline_number, 1):
        if Pc_r[i] > Pd1 :
        
            BVOCC = (BVocc_r[i])

            bvarray_pore1.append(BVocc_r[i]); #add items 
            pcarray_pore1.append(Pc_r[i]); #add items     


    ydata=np.array(bvarray_pore1)
    xdata=np.array(pcarray_pore1)
    
    # Define BV1 for Single Pore System rock
    #BV1 = np.max(bvarray_pore1)
    
    def func1(xdata, a, b, c):
        return a*10**((-0.434*b)/np.log10(xdata/c))

    # No exception Exception raised in try block
    try:
        popt, pcov = curve_fit(func1, xdata, ydata, method='trf', bounds=([1.0, .1, 1.0 ], [np.inf, np.inf, np.inf]))

        BV1_solve = popt[0]   
        G1_solve  = popt[1]
        Pd1_solve = popt[2]
        quality1 = "GOOD FIT"
        print()
        print('Good Curve Fit for Pore System 1 of 2 Pore Systems',', Quality =',quality)
        print()
        #print('      Pd1 pick =',round(Pd1,1),', G1 or popt[1] =',round(popt[1],2),',       BV1 pick =',round(BV1,2))
        print('Pd1 or popt[2] =',round(Pd1_solve,1),', G1 or popt[1] =',round(G1_solve,2),', BV1 or popt[0] =',round(BV1_solve,2))
        print()

    # handles curve_fit issues  
    except :
        # this block is always executed 
        quality1  = "BRUTE FORCE SYS1/1"
        BV1_solve = BV1 
        Pd1_solve = Pd1

        # =============================================================================
        #             Brute Strength error minimization for G1_Solve
        # ============================================================================= 
        error=999
        g=0.01
        
        for i in range (0,30,1) :
            # =============================================================================
            #             Calculate Errror Between BVocc
            # =============================================================================
            bvarray_for_error = []
            for i in range(0, spline_number, 1):
                if Pc_r[i] > Pd1_solve:
                    BVOCC1 = BV1_solve * 10**((-0.434 * g) / np.log10(Pc_r[i] / Pd1_solve))
                else:
                    BVOCC1 = 0.001

                BVOCC = BVOCC1

                bvarray_for_error.append(BVOCC); #add items 

            squared_array = np.square(np.subtract(BVocc_r, bvarray_for_error))
            error_bvocc = squared_array.mean()

            if error_bvocc < error:
                G1_solve=round(g,2)
                error=error_bvocc

            g = g + 0.01

            #print('G1_solve =',round(G1_solve,2), ', Error =', round(error,4))
        print(len(BVocc_r),len(bvarray_for_error))
        print()
        print('G1_solve =',G1_solve, ', Error =', round(error,4))

        # regardless of exception generation.
        print() 
        print('Brute Force Fit for Pore System 1, and only 1 Pore System',', Quality =',quality1)
        print()
        #print('Pd1 pick =',round(Pd1,1),', G1 - assigned as 0.51???',',       BV1 pick =',round(BV1,2))
        print()

        Pd2_solve = float("nan")
        G2_solve  = float("nan")
        BV2_solve = float("nan")

        BVtotal = BV1_solve 

    # =============================================================================
    #             Create Pc Curves for curve fit Thomeer parameters
    # =============================================================================
    Pc = 0.5
    bvarray_pred = []; #make list of 0 length
    pcarray_pred = []

    for j in range(0, 105, 1):
        if Pc > Pd1_solve:
            BVOCC1 = BV1_solve * 10**((-0.434 * G1_solve) / np.log10(Pc / Pd1_solve))
        else:
            BVOCC1 = 0.001


        BVOCC = BVOCC1

        bvarray_pred.append(BVOCC); #add items 
        pcarray_pred.append(Pc); #add items 

        Pc = Pc * 1.12


    x_solve=np.array(bvarray_pred)
    y_solve=np.array(pcarray_pred)



In [8]:
# =============================================================================
#             Scipy for Curve_fit of 2 Pore Systems
# =============================================================================
if no_pore_sys == 2  :
    
    # Find Pd1
    Pd1_min_element = min(min(np.where(BVocc_r>0.1)))
    Pd1=Pc_r[Pd1_min_element]

    bvarray_pore1 = []; #make list of 0 length
    pcarray_pore1 = []

    for i in range(Pd1_min_element, spline_number, 1):
        if  Pc_r[i] < Pd2 :
            BVOCC = (BVocc_r[i])

            bvarray_pore1.append(BVocc_r[i]); #add items 
            pcarray_pore1.append(Pc_r[i]); #add items     

    ydata=np.array(bvarray_pore1)
    xdata=np.array(pcarray_pore1)
    
    # Determine BV1
    #BV1 = np.max(bvarray_pore1)
    
    
    def func1(xdata, a, b, c):
        return a*10**((-0.434*b)/np.log10(xdata/c))

     # No exception Exception raised in try block
    try:
        popt, pcov = curve_fit(func1, xdata, ydata, method='trf', bounds=([1.0, .05, 1.0 ], [np.inf, np.inf, np.inf]))

        BV1_solve = popt[0]   
        G1_solve  = popt[1]
        Pd1_solve = popt[2]
        quality1 = "GOOD FIT SYS1/2"
        print()
        print('Good Curve Fit for Pore System 1 of 2 Pore Systems',', Quality =',quality1)
        print()
        print('      Pd1 pick =',round(Pd1,1),', G1 or popt[1] =',round(popt[1],2),',       BV1 pick =',round(BV1,2))
        print('Pd1 or popt[2] =',round(Pd1_solve,1),', G1 or popt[1] =',round(G1_solve,2),', BV1 or popt[0] =',round(BV1_solve,2))
        print()


    # handles issues  
    except :
        # this block is always executed 
        quality1 = "BRUTE FORCE SYS1/2"
        BV1_solve = BV1  
        Pd1_solve = Pd1

        # =============================================================================
        #             Brute Strength error minimization for G1_Solve
        # ============================================================================= 
        error=999
        g=0.01
        
        for i in range (0,30,1) :
            # =============================================================================
            #             Calculate Errror Between BVocc
            # =============================================================================
            bvarray_for_error = []
            for i in range(0, spline_number, 1):
                if Pc_r[i] > Pd1_solve:
                    BVOCC1 = BV1_solve * 10**((-0.434 * g) / np.log10(Pc_r[i] / Pd1_solve))
                else:
                    BVOCC1 = 0.001

                BVOCC = BVOCC1

                bvarray_for_error.append(BVOCC); #add items 

            squared_array = np.square(np.subtract(BVocc_r, bvarray_for_error))
            error_bvocc = squared_array.mean()

            if error_bvocc < error:
                G1_solve=round(g,2)
                error=error_bvocc

            g = g + 0.01

            #print('G1_solve =',round(G1_solve,2), 'Error =', round(error,4))

        print()
        print('G1_solve =',G1_solve, 'Error =', round(error,4))

        # regardless of exception generation.
        print() 
        print('Brute Force Fit for Pore System 1 of 2 Pore Systems',', Quality =',quality1)
        print()
        print('Pd1_solve  =',round(Pd1_solve,1),', G1_solve =',round(G1_solve,2),', BV1_solve =',round(BV1_solve,2))
        print()


    # =============================================================================
    #             Scipy for Curve_fit of Pd2, G2 and BV2
    # =============================================================================
    bvarray_pore2 = []; #make list of 0 length
    pcarray_pore2 = []

    for i in range(0, spline_number, 1):
        if Pc_r[i] > Pd2 :

            BVOCC = (BVocc_r[i])

            bvarray_pore2.append(BVocc_r[i]); #add items 
            pcarray_pore2.append(Pc_r[i]); #add items     


    ydata2=np.array(bvarray_pore2)
    xdata2=np.array(pcarray_pore2)
    
    # Determine BVtotal and then BV2
    #BVtotal = np.max(bvarray_pore2)
    BV2 = BVtotal - BV1
    
    def func2(xdata2, a, b, c):
        return a*10**((-0.434*b)/np.log10(xdata2/c))

    #popt, pcov = curve_fit(func2, xdata2, ydata2, method='dogbox', bounds=([0.0 , 0.1, 1.0 ], [np.inf, np.inf, np.inf]))    

    # No exception Exception raised in try block
    try:
        popt, pcov = curve_fit(func2, xdata2, ydata2, method='dogbox', bounds=([0.0 , 0.01, 1.0 ], [np.inf, np.inf, np.inf]))
        quality2  = "GOOD FIT SYS2/2"        
        print()
        print('Good Curve Fit for Pore System 2 of 2 Pore Systems',', Quality =',quality2)

        #BV2_solve = popt[0] - BV1_solve
        ####if  popt[0] - BV1_solve > 0:
        ####    BV2_solve = popt[0] - BV1_solve   
        ####else:
        ####    BV2_solve = 0      
        BV2_solve = BV2

        G2_solve  = popt[1]
        #Pd2_solve = popt[2]
        Pd2_solve = Pd2

        print()
        print('Pd2_solve  =',round(Pd2_solve,1),', G2_solve =',round(G2_solve,2),', BV2_solve =',round(BV2_solve,2))
        print()


    # handles issues  
    except :
            
        # this block is always executed 
        BV2_solve = BV2   
        G2_solve  = .1
        Pd2_solve = Pd2
        quality2  = "BAD FIT SYS2/2"
        # regardless of exception generation.
        print() 
        print('Bad Curve Fit for Pore System 2 of 2 Pore Systems',', Quality =',quality2)
        print()
        print('Pd2  =',round(Pd2_solve,1),', G2 or popt[1] =',round(G2_solve,2),', BV2 or popt[0] =',round(BV2_solve,2))
        print()



    # =============================================================================
    #             Create Pc Curves for curve fit Thomeer parameters
    # =============================================================================
    Pc = 0.5
    bvarray_pred = []; #make list of 0 length
    pcarray_pred = []
    bvocc2_pred  = []

    for j in range(0, 105, 1):
        if Pc > Pd1_solve:
            BVOCC1 = BV1_solve * 10**((-0.434 * G1_solve) / np.log10(Pc / Pd1_solve))
        else:
            BVOCC1 = 0.001

        if Pc > Pd2_solve:
            BVOCC2 = BV2_solve * 10**((-0.434 * G2_solve) / np.log10(Pc / Pd2_solve))
        else:
            BVOCC2 = 0.001


        BVOCC = BVOCC1 + BVOCC2

        bvarray_pred.append(BVOCC); #add items 
        pcarray_pred.append(Pc); #add items 
        bvocc2_pred.append(BVOCC2)  

        Pc = Pc * 1.12


    x_solve=np.array(bvarray_pred)
    x2_solve=np.array(bvocc2_pred)
    y_solve=np.array(pcarray_pred)
    # =============================================================================
    #             End of Scipy for Curve_fit of Pd1, G1 and BV1
    # =============================================================================



Good Curve Fit for Pore System 1 of 2 Pore Systems , Quality = GOOD FIT SYS1/2

      Pd1 pick = 9.9 , G1 or popt[1] = 0.51 ,       BV1 pick = 7.39
Pd1 or popt[2] = 8.3 , G1 or popt[1] = 0.51 , BV1 or popt[0] = 10.0


Good Curve Fit for Pore System 2 of 2 Pore Systems , Quality = GOOD FIT SYS2/2

Pd2_solve  = 426.5 , G2_solve = 0.33 , BV2_solve = 3.47



## Calculate Mode of PTD, Thomeer Perm and Calculate Error to the Thomeer Analysis:

In [9]:
mode = np.exp(-1.15*G1_solve)*(214/Pd1_solve)
perm_thomeer = (3.8068*G1_solve**(-1.334))*((BV1_solve/Pd1_solve)**2)


# =============================================================================
#             Calculate Errror Between BVocc
# =============================================================================
bvarray_for_error = []

if no_pore_sys == 1: 
    
    for i in range(0, spline_number, 1):
        if Pc_r[i] > Pd1_solve:
            BVOCC1 = BV1_solve * 10**((-0.434 * G1_solve) / np.log10(Pc_r[i] / Pd1_solve))
        else:
            BVOCC1 = 0.001

        BVOCC = BVOCC1

        bvarray_for_error.append(BVOCC); #add items 
    
elif no_pore_sys > 1: 
    
    for i in range(0, spline_number, 1):
        if Pc_r[i] > Pd1_solve:
            BVOCC1 = BV1_solve * 10**((-0.434 * G1_solve) / np.log10(Pc_r[i] / Pd1_solve))
        else:
            BVOCC1 = 0.001
    
        if Pc_r[i] > Pd2_solve:
            BVOCC2 = BV2_solve * 10**((-0.434 * G2_solve) / np.log10(Pc_r[i] / Pd2_solve))
        else:
            BVOCC2 = 0.001

        BVOCC = BVOCC1 + BVOCC2
    
        bvarray_for_error.append(BVOCC); #add items 

print(len(Pc_r), len(BVocc_r),  len(bvarray_for_error))

#error_bvocc = np.linalg.norm(BVocc_r - bvarray_forerror) / np.linalg.norm(bvarray_forerror)

difference_array = np.subtract(BVocc_r, bvarray_for_error)
squared_array = np.square(difference_array)
error_bvocc = squared_array.mean()

print('Mode of PTD =', round(mode,2), ', Perm_Thomeer =',round(perm_thomeer,2), ', Error BVocc or MSE =', round(error_bvocc,3))

400 400 400
Mode of PTD = 14.33 , Perm_Thomeer = 13.54 , Error BVocc or MSE = 0.077


In [10]:
# =============================================================================
#             Final Plot
# =============================================================================
plt.figure(figsize=(10,12))
#ax.loglog(x2, y2, 'r-'  , linewidth=3 , label='Nearest Pc Curve')
plt.loglog(x_Pc_nocc, y_Pc, 'k--' , linewidth=1 ,markersize = 1, label='Actual HPMI Pc Curve')
plt.loglog(Closure, Pd1, 'ko' , linewidth=10 , label='Closure Correction')        
#ax.loglog(x5, y5, 'b--' , linewidth=3 , label='kNN Pc Curve')
plt.loglog(x_Pc, y_Pc, 'g-' , linewidth=2 , label='Closure Corrected Pc Curve')
#ax.loglog(x_gui  , y_gui,   'b--' , linewidth=1 , label='Pc from GUI Picks')
plt.loglog(x_solve, y_solve, 'r-' , linewidth=3 , label='Thomeer Modeled Pc Curve')

       
plt.xlim(50, 0.1)
#ax.gca().invert_xaxis()
plt.ylim(1, 100000)
#ax.set_title("Pc Curves from Scipy Curve_fit", fontname="Times New Roman", size=24,fontweight="bold", color='blue')            
plt.title("Thomeer Parameters from Scipy Curve_fit used to Model HPMI data", size=16,fontweight="bold", color='blue')            

plt.ylabel('Pc Hg', fontsize=16, fontweight="bold", color = 'blue')
plt.xlabel('BVOCC', fontsize=16, fontweight="bold", color = 'blue')
plt.grid(True, which="both",ls="-")
plt.legend()

plt.text(50,8,' h = 2.4ft'  ,horizontalalignment='left', fontsize=10, fontweight="bold",color='green')
plt.text(50,80,' h = 24.5ft',horizontalalignment='left', fontsize=10,fontweight="bold", color='green')
plt.text(50,800,' h = 245ft',horizontalalignment='left', fontsize=10,fontweight="bold", color='green') 
plt.text(50,Pd1,'---- height @ Pd',horizontalalignment='left', fontsize=12, fontweight="bold", color='blue', fontstyle='italic') 

plt.text(.1,Pd1,' Pd1',horizontalalignment='left', size=14, fontweight="bold", color='blue')
#plt.text(max(diff(x_Pc)) + 1,    Pd1 + 6*Pd1,'    G1',horizontalalignment='right',  size=14,fontweight="bold", color='blue')
plt.text(BV1_solve + BV2_solve +4,  14000,'  BV_infinite',horizontalalignment='right',  size=14,fontweight="bold", color='blue')
plt.axvline(x= BVtotal+1, color='blue' , linestyle='--')  #vertical line        

#plt.annotate('            ', fontsize=14, color='blue', xy=(BV1_solve + BV2_solve +0, 10000), xytext=(40,12000),
#    arrowprops=dict(facecolor='blue', shrink=0.05),
#)

plt.text(40,3.3   , 'Thomeer Parameter Estimates from Scipy curve_fit:', horizontalalignment='left', fontsize=14, fontweight="bold",color='blue')
plt.text(.35,4.2  , quality1, horizontalalignment='left', fontsize=10, fontweight="bold",color='red')
plt.text(.35,3.3  , quality2, horizontalalignment='left', fontsize=10, fontweight="bold",color='red')


plt.text(40,2.2  , 'Pd1 ='   , horizontalalignment='left', fontsize=12, fontweight="bold",color='red')
plt.text(20,2.2  , round(Pd1_solve,1) , horizontalalignment='left', fontsize=12, fontweight="bold",color='red')
plt.text(6,2.2   , 'G1 ='  , horizontalalignment='left', fontsize=12, fontweight="bold",color='red')
plt.text(3.5,2.2   , round(G1_solve,3)  , horizontalalignment='left', fontsize=12, fontweight="bold",color='red')
plt.text(0.8,2.2 ,'BV1 = ' , horizontalalignment='left', fontsize=12, fontweight="bold",color='red')
plt.text(0.4,2.2 , round(BV1_solve,2) , horizontalalignment='left', fontsize=12, fontweight="bold",color='red')


if no_pore_sys > 1:     
    plt.loglog(x2_solve, y_solve, 'k--' , linewidth=1 , label='Thomeer Modeled BVOCC2')
    plt.text(40,1.5  , 'Pd2 ='   , horizontalalignment='left', fontsize=12, fontweight="bold",color='brown')
    plt.text(20,1.5  , round(Pd2_solve,1) , horizontalalignment='left', fontsize=12, fontweight="bold",color='brown')
    plt.text(6,1.5   , 'G2 ='  , horizontalalignment='left', fontsize=12, fontweight="bold",color='brown')
    plt.text(3.5,1.5   , round(G2_solve,3)  , horizontalalignment='left', fontsize=12, fontweight="bold",color='brown')
    plt.text(0.8,1.5 ,'BV2 = ' , horizontalalignment='left', fontsize=12, fontweight="bold",color='brown')
    plt.text(0.4,1.5 , round(BV2_solve,2) , horizontalalignment='left', fontsize=12, fontweight="bold",color='brown')

plt.text(40,1.1  , 'Sample_NO ='   , horizontalalignment='left', fontsize=12, fontweight="bold",color='black')
plt.text(14,1.1  , sample_no    , horizontalalignment='left', fontsize=12, fontweight="bold",color='black')
plt.text(6,1.1 ,'C_Correction  = ' , horizontalalignment='left', fontsize=12, fontweight="bold",color='black')
plt.text(1.7,1.1 , round(Closure,2) , horizontalalignment='left', fontsize=12, fontweight="bold",color='black')
plt.text(0.8,1.1 ,'MSE  = ' , horizontalalignment='left', fontsize=12, fontweight="bold",color='black')
plt.text(0.4,1.1 , round(error_bvocc,3) , horizontalalignment='left', fontsize=12, fontweight="bold",color='black')


plt.show()

g1_solve  = G1_solve
g2_solve  = G2_solve
pd1_solve = Pd1_solve
pd2_solve = Pd2_solve
bv1_solve = BV1_solve
bv2_solve = BV2_solve
closure   = Closure

 


# Store data into geolog
#geolog.puttable()

#geolog.putrow()





# This section for creating a header and saving Thomeer parameters for a file is work in progress. 

Write Header to file first:

In [11]:
filename = './data/Thomeer_results.xlsx'
wb = openpyxl.load_workbook(filename=filename)
sheet = wb['Sheet1']
header = ['Pd1', 'G1', 'BV1', 'Pd2', 'G2', 'BV2']

sheet.append(header)

wb.save(filename)

In [12]:
filename = './data/Thomeer_results.xlsx'
wb = openpyxl.load_workbook(filename=filename)
sheet = wb['Sheet1']
new_row = [Pd1_solve, G1_solve, BV1_solve, Pd2_solve, G2_solve, BV2_solve]

sheet.append(new_row)

wb.save(filename)
